In [1]:
from mpcbinning.controllers.controller import *
from mpcbinning.utils.data_manipulation import *
from databinning.plants.Plant import GaussianPlant
from databinning.models.ToggleSwitch import ToggleSwitchSimplified
import gc
import sys

In [2]:
x_dim = 2
u_dim = 2

x_min = np.zeros(x_dim)
u_min = np.zeros(u_dim)

x_max = np.array([150, 100])
u_max = np.array([35, 0.35])

x_discr = np.array([300, 200])
u_discr = np.array([35, 35])

x_step = np.array((x_max - x_min)/x_discr)
u_step = np.array((u_max - u_min)/u_discr)

In [ ]:
x1 = np.arange(x_min[0], x_max[0] + 2, 1)
x2 = np.arange(x_min[1], x_max[1] + 2, 1)

x_comb = np.array(np.meshgrid(x1, x2)).T.reshape(-1, 2)
n_comb = x_comb.shape[0]
print(n_comb)

del x1, x2
gc.collect()

In [ ]:
total_time = 100
t_step = 1
steps = int(total_time/t_step)

In [ ]:
setup_mpc = {
    'n_robust': 1,
    'n_horizon': 5,
    't_step': t_step,
    'store_full_solution': False
}

model = template_model(True)
mpc = template_mpc(model, setup_mpc)
simulator = template_simulator(model, t_step)
estimator = do_mpc.estimator.StateFeedback(model)

In [ ]:
%%capture --no-stderr

trajectories = [0]*n_comb

for i in range(n_comb):
    print("Iteration: " + str(i), file=sys.stderr)

    mpc.reset_history()
    simulator.reset_history()
    estimator.reset_history()

    x_0 = x_comb[i]

    mpc.x0 = x_0
    simulator.x0 = x_0
    estimator.x0 = x_0
    
    mpc.set_initial_guess()

    for k in range(steps):
        u = mpc.make_step(x_0)
        y_next = simulator.make_step(u, np.random.randn(model.n_v, 1), np.random.randn(model.n_w, 1))
        x_0 = estimator.make_step(y_next)

    x = np.array([x for x in mpc.data['_x'].tolist()])
    u = np.array([x for x in mpc.data['_u'].tolist()])    
    trajectories[i] = np.array([x, u]) 
    del x, u
    gc.collect()

del model, mpc, simulator, estimator, x_comb
gc.collect()

In [ ]:
np.save('./data/fx_trajectories.npy', trajectories)

In [ ]:
plant = GaussianPlant(x_dim, x_discr, u_discr)
f_x = plant.getPlant(trajectories, x_dim, u_dim, x_step, u_step, x_max, u_max, x_min, u_min, np.array([x_step[0], x_step[1]]))
np.save('./data/f_x.npy', f_x)
